In [1]:
import torch
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_xs', pretrained=True)
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_s', pretrained=True)
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_m', pretrained=True)

Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master
Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master
Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master


In [7]:
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data import DistributedSampler, RandomSampler, SequentialSampler


from torchvision import transforms


from pytorchvideo.models import x3d
from pytorchvideo.data import (
    Ucf101, 
    RandomClipSampler, 
    UniformClipSampler, 
    Kinetics,
    SSv2
)


from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
)


import torchinfo

from tqdm.notebook import tqdm
import itertools
import os
import pickle

In [8]:
class Args:
    def __init__(self):
        self.metadata_path = '/mnt/NAS-TVS872XT/dataset/'
        self.root = self.metadata_path
        self.annotation_path = self.metadata_path
        self.FRAMES_PER_CLIP = 16
        self.STEP_BETWEEN_CLIPS = 16
        self.BATCH_SIZE = 16
        self.NUM_WORKERS = 8  # kinetics:8, ucf101:24
        # self.CLIP_DURATION = 16 / 25
        self.CLIP_DURATION = (8 * 8) / 30  # (num_frames * sampling_rate)/fps
        self.VIDEO_NUM_SUBSAMPLED = 16  # 事前学習済みモデルに合わせて16→8
        self.UCF101_NUM_CLASSES = 101
        self.KINETIC400_NUM_CLASSES = 400
        self.MODEL_NAME = "x3d_m"

args = Args()

In [9]:
class LimitDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        self.dataset_iter = itertools.chain.from_iterable(
            itertools.repeat(iter(dataset), 2)
        )

    def __getitem__(self, index):
        return next(self.dataset_iter)

    def __len__(self):
        return self.dataset.num_videos

In [10]:
def get_kinetics(subset):
    """
    Kinetics400のデータセットを取得

    Args:
        subset (str): "train" or "val" or "test"

    Returns:
        pytorchvideo.data.labeled_video_dataset.LabeledVideoDataset: 取得したデータセット
    """
    args = Args()
    transform = Compose([
        ApplyTransformToKey(
            key="video",
            transform=Compose([
                UniformTemporalSubsample(args.VIDEO_NUM_SUBSAMPLED),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ShortSideScale(size=256),
                # RandomShortSideScale(min_size=256, max_size=320,),
                # CenterCropVideo(crop_size=(256, 256)),
                CenterCrop(224),
                # RandomCrop(224),
                RandomHorizontalFlip(),
            ]),
        ),
        ApplyTransformToKey(
            key="label",
            transform=transforms.Lambda(lambda x: x),
        ),
        RemoveKey("audio"),
    ])

    root_kinetics = '/mnt/NAS-TVS872XT/dataset/Kinetics400/'

    if subset == "test":
        dataset = Kinetics(
            data_path=root_kinetics + "test_list.txt",
            video_path_prefix=root_kinetics + 'test/',
            clip_sampler=RandomClipSampler(clip_duration=args.CLIP_DURATION),
            video_sampler=RandomSampler,
            decode_audio=False,
            transform=transform,
        )
        return dataset
    else:
        dataset = Kinetics(
            data_path=root_kinetics + subset,
            video_path_prefix=root_kinetics + subset,
            clip_sampler=RandomClipSampler(clip_duration=args.CLIP_DURATION),
            video_sampler=RandomSampler,
            decode_audio=False,
            transform=transform,
        )
        return dataset

    return False

In [11]:
def make_loader(dataset):
    """
    データローダーを作成

    Args:
        dataset (pytorchvideo.data.labeled_video_dataset.LabeledVideoDataset): get_datasetメソッドで取得したdataset

    Returns:
        torch.utils.data.DataLoader: 取得したデータローダー
    """
    args = Args()
    loader = DataLoader(LimitDataset(dataset),
                        batch_size=args.BATCH_SIZE,
                        drop_last=True,
                        num_workers=args.NUM_WORKERS)
    return loader

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_org = torch.hub.load(
        'facebookresearch/pytorchvideo', args.MODEL_NAME, pretrained=True)
model_org = model_org.to(device)
dataset = get_kinetics("test")

Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master


In [13]:
data = dataset.__next__()
data = data["video"]
print(data.shape)

torch.Size([3, 16, 224, 224])


In [15]:
batch_size = 1

torchinfo.summary(
    model_org,
    input_size=(batch_size,3,16,224,224),
    depth=6,
    col_names=["input_size",
               "output_size"],
    row_settings=("var_names",)
)

Layer (type (var_name))                                      Input Shape               Output Shape
Net                                                          --                        --
├─ModuleList (blocks)                                        --                        --
│    └─ResStage (1)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --
│    └─ResStage (2)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --
│    └─ResStage (3)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --
│    └─ResStage (4)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --


In [22]:
torchinfo.summary(
    model_org.blocks,
    # input_size=(batch_size,3,16,224,224),
    depth=6,
    # col_names=["input_size",
    #            "output_size"],
    row_settings=("var_names",)
)

Layer (type (var_name))                                 Param #
ModuleList                                              --
├─ResNetBasicStem (0)                                   --
│    └─Conv2plus1d (conv)                               --
│    │    └─Conv3d (conv_t)                             648
│    │    └─Conv3d (conv_xy)                            120
│    └─BatchNorm3d (norm)                               48
│    └─ReLU (activation)                                --
├─ResStage (1)                                          --
│    └─ModuleList (res_blocks)                          --
│    │    └─ResBlock (0)                                --
│    │    │    └─Conv3d (branch1_conv)                  576
│    │    │    └─BottleneckBlock (branch2)              --
│    │    │    │    └─Conv3d (conv_a)                   1,296
│    │    │    │    └─BatchNorm3d (norm_a)              108
│    │    │    │    └─ReLU (act_a)                      --
│    │    │    │    └─Conv3d (conv_b)       

In [56]:
class ReconstructNet(nn.Module):
    def __init__(self):
        super().__init__()
        model = torch.hub.load(
        'facebookresearch/pytorchvideo', args.MODEL_NAME, pretrained=True)

        self.net_bottom = nn.Sequential(
            model.blocks[0],
            model.blocks[1],
            model.blocks[2],
            model.blocks[3],
            model.blocks[4]
        )

        self.net_top = nn.Sequential(
            model.blocks[5].pool,
            model.blocks[5].dropout
        )

        self.linear = model.blocks[5].proj

        self.net_output = nn.Sequential(
            model[5].activation,
            model[5].output_pool
            # nn.Softmax
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.net_bottom(x)
        x = self.net_top(x)
        x = x.permute(0,2,3,4,1)
        x = self.linear(x)
        x = x.permute(0,4,1,2,3)
        x = self.net_output(x)
        return x


model_new = ReconstructNet()

Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master


TypeError: 'Net' object is not subscriptable

In [54]:
model_new = ReconstructNet()
model_new = model_new.to(device)

Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master


In [55]:
torchinfo.summary(
    model_new,
    input_size=(batch_size,3,16,224,224),
    depth=4,
    col_names=["input_size",
               "output_size"],
    row_settings=("var_names",)
)

Layer (type (var_name))                                      Input Shape               Output Shape
ReconstructNet                                               --                        --
├─Sequential (net_bottom)                                    --                        --
│    └─ResStage (1)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --
│    └─ResStage (2)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --
│    └─ResStage (3)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --
│    └─ResStage (4)                                          --                        --
│    │    └─ModuleList (res_blocks)                          --                        --
